### High quality code
We will talk about two important software quality metrics in this notebook that will help us improve our code quality & allows us to extend or change our code easily in future. Those two metrics are:
- Cohesion
- Coupling

## Cohesion
- Cohesion is degree to which elements of a certain class or function belong together.

Let's see some code snippet to understand it better.

- The following function `func1()` does a lot of different unrelated things which means that it has a weak cohesion.
<pre>
def func1(some_args):
    # Downloads a html file
    # Downloads an image
    # Sends both files to an email
    # Calculates hash of html file
    # Returns success_msg and hash of file
</pre>

- Similarly, the following function `func2()` has a single function i.e. to add numbers thus holds a strong cohesion.
<pre>
def func1(some_args):
    # Add numbers
    # Return the sum
</pre>

- To maintain, change or extend the code easily, a function should have `strong cohesion`

## Coupling
- Coupling is measure of how dependent two parts of our code are to each other.

Let's see some code snippet to understand it better.

- Let's look at the following function: `checkEmailSec()` 
<pre>
def checkEmailSec(email):
    if email.header.bearer.invalid():
        raise Exception("Email header bearer is invalid")
    elif email.header.received != email.header.received_spf:
        raise Exception("Received mismatch")
    else:
        print("Email header is secure")_
</pre>

- This function checks an email if its legit or not by looking at various parts of email.
- The code in function directly accesses the data that is deep in the structure of `email` object which means that its highly coupled with email object.
- `Having high` is problematic because changing something in one part of our program requires a change in multiple places of program. Like in this example, if the structure of email changes we have to rewrite the `checkEmailSec()` function as well.
- Although, there will always be some coupling in our code because there will always be some code that will be dependent to each other.
- Know that, if you introduce more coupling to the code, it will become difficult to change the code.

- We can solve the coupling issue in the above example by either just passing the data that the function needs (header) instead of entire email object or we can move this function inside Email object since its directly related to the data in object.


`Coupling & Cohesion are the important metrics for software quality. Unfortunately, there is no formula that can help us calculate a number that tells us how much two functions/classes are coupled or cohesive. We have to train ourselves daily to master this skill.`

We will see an examples to understand better.

In [1]:
import string
import random

In [2]:
class VehicleRegistry:
    def generate_vehicle_id(self,length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))
    
    def generate_vehicle_license(self,id):
        return f"{id[:2]}-{''.join(random.choices(string.digits,k=2))}-{''.join(random.choices(string.ascii_uppercase,k=2))}"

In [3]:
class Application:
    def register_vehicle(self,brand):
        # Create a registry instance
        registry = VehicleRegistry()
        
        #generate a vehicle id of length 12
        vehicle_id = registry.generate_vehicle_id(12)
        
        #generate  a license plate for vehicle
        license_plate = registry.generate_vehicle_license(vehicle_id)
        
        # computer catalog price
        catalog_price = 0
        if brand == "Tesla Model 3":
            catalog_price = 10000
        elif brand == "Volkswagen ID3":
            catalog_price = 20000
        elif brand == "BMW 5":
            catalog_price = 15000
            
        # Compute tax percentage
        tax_percentage = 0.05
        if brand == "Tesla Model 3" or "Volkswagen ID3":
            tax_percentage = 0.02
        
        # Computer payable tax
        payable_tax = catalog_price * tax_percentage
        
        # Print results
        print("Registration Completed. Vehicle Info:")
        print(f"Brand: {brand}")
        print(f"ID: {vehicle_id}")
        print(f"License Plate: {license_plate}")
        print(f"Payable Tax: {payable_tax}")

# Create instance
app = Application()
app.register_vehicle("BMW 5")
        

Registration Completed. Vehicle Info:
Brand: BMW 5
ID: KAKTWNJOBKOI
License Plate: KA-47-UI
Payable Tax: 300.0


Lets figure out the problems with our code & then apply the improvement to make it better. 

- `register_vehicle()` is doing a lot of different things i.e 
    - It generates ID & license plate
    - It computes catalogue price & tax
    - It prints out the results
    - It has a very low cohesion. 
    - It also has a high coupling i.e. this function is directly relying on implementation details of `VehicleRegistry`
    - High coupling means that if we change anything in `VehicleRegistry`, we have to change couple of things in `register_vehicle()` method as well.
    - Also, it is hard to add more brands and catalogue prices as its forcing us to add more `if else` statements.
    - Same is the case with `tax` generation code.
    

<b>TIP: </b> `TO IMPROVE COHESION & COUPLING,ALWAYS LOOK FOR THE INFORMATION THAT THE APPLICATION IS USING`

Lets make this code better and easy to use.

In [4]:
# Lets divide the info into two classes
class VehicleInfo:
    brand: str
    catalogue_price: int
    electric: bool
        
    ## Initializer
    def __init__(self,brand,electric,catalogue_price):
        self.brand = brand
        self.catalogue_price = catalogue_price
        self.electric = electric
        
    # Most logical place to put tax calculation method
    def compute_tax(self):
        tax_percentage = 0.05
        if self.electric:
            tax_percentage = 0.02
        
        return tax_percentage * self.catalogue_price
    
    def print(self):
        print(f"Brand: {self.brand}")
        print(f"Payable tax: {self.compute_tax()}")
            
        
class Vehicle:
    id: str
    license_plate: str
    info: VehicleInfo
    
    ## Initializer
    def __init__(self,id,license_plate,info):
        self.id = id
        self.license_plate = license_plate
        self.info = info
        
    def print_vehicle_info(self):
        print(f"ID: {self.id}")
        print(f"License Plate: {self.license_plate}")
        self.info.print()

In [5]:
import random
import string
# Now let's move Vehicle registry related info to VehicleRegistry class
class VehicleRegistry:
    vehicle_info = {}
    
    def __init__(self):
        self.add_vehicle_info("Tesla Model 3",True,560000)
        self.add_vehicle_info("Volkswagen ID3",True,420000)
        self.add_vehicle_info("BMW 5",False,30000)
        
    
    def add_vehicle_info(self,brand,electric,catalogue_price):
        self.vehicle_info[brand] = VehicleInfo(brand,electric,catalogue_price)
    
    def generate_vehicle_id(self,length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))
    
    def generate_vehicle_license(self,id):
        return f"{id[:2]}-{''.join(random.choices(string.digits,k=2))}-{''.join(random.choices(string.ascii_uppercase,k=2))}"
    
    def create_vehicle(self,brand):
         #generate a vehicle id of length 12
        vehicle_id = self.generate_vehicle_id(12)
        #generate  a license plate for vehicle
        license_plate = self.generate_vehicle_license(vehicle_id)
        return Vehicle(id=vehicle_id,license_plate=license_plate,info=self.vehicle_info[brand])
        

In [6]:
class Application:
    def register_vehicle(self,brand: str):
        # Create registry
        registry = VehicleRegistry()
        # Create a vehicle 
        return registry.create_vehicle(brand)

In [7]:
app = Application()
vehicle = app.register_vehicle("Volkswagen ID3")
# Printing the information
vehicle.print_vehicle_info()

ID: CEARXWFRDKRQ
License Plate: CE-90-RD
Brand: Volkswagen ID3
Payable tax: 8400.0


- Now we can see that every variable and every function resides where it should be
- Its much easier to extend this code in future as compared to what we wrote earlier.
`Although, there is are more improvements we can do but for the sake of understanding COHESION & COUPLING its more than enough`